Drawing tools are the basis of a wide range of functionality in EarthSim allowing for annotating existing data, highlighting regions of interest and drawing and editing shapes. This user guide will give a basic introduction to the drawing tools, explaining how to sync the data back to Python and how to access the data. For more detail about the underlying bokeh tools [see the bokeh user guide](https://bokeh.pydata.org/en/latest/docs/user_guide/tools.html#userguide-tools-edit).

In [ ]:
import numpy as np
import holoviews as hv
import geoviews as gv
import cartopy.crs as ccrs

from holoviews.streams import (
    PointDraw, PolyEdit, BoxEdit, PolyDraw
)

url = 'http://c.tile.openstreetmap.org/{Z}/{X}/{Y}.png'
tiles = gv.WMTS(url, extents=(-91, 32.2, -90.8, 32.4), crs=ccrs.PlateCarree())

hv.extension('bokeh')

## Drawing Points

All drawing tools are added via a corresponding stream, which also syncs the data. Here we will use the ``PointDraw`` stream which allows adding points, dragging points and deleting points.

**Add point**

    Tap anywhere on the plot

**Move point**
    
    Tap and drag an existing point, the point will be dropped once you let go of the mouse button.

**Delete point**

    Tap a point to select it then press BACKSPACE key while the mouse is within the plot area.

In [ ]:
%%opts Points [width=900 height=500 tools=['hover']] (size=10 color='red')
points = gv.Points(np.random.rand(10, 2)*2000000, crs=ccrs.GOOGLE_MERCATOR)
point_stream = PointDraw(source=points)
tiles * points

Once you have drawn a point on the map we can wrap it in a GeoViews Points object, project it and display the points as a dataframe:

In [ ]:
if point_stream.data:
    projected = gv.operation.project(point_stream.element, projection=ccrs.PlateCarree())
    display(projected.dframe())

## Drawing bounding boxes

The ``BoxEdit`` stream adds a tool which allows drawing, dragging and deleting rectangular bounding boxes:

**Add box**

    Hold shift then click and drag anywhere on the plot.

**Move box**

    Click and drag an existing box, the box will be dropped once you let go of the mouse button.

**Delete box**

    Tap a box to select it then press BACKSPACE key while the mouse is within the plot area.

In [ ]:
%%opts Polygons [width=900 height=500] (fill_alpha=0 line_color='black' selection_fill_color='red')
box_poly = gv.Polygons([], crs=ccrs.GOOGLE_MERCATOR)
box_stream = BoxEdit(source=box_poly)
tiles * box_poly

Now we can get the bounding box from the stream and use the GeoViews ``project_extents`` utility to project it from Mercator to lats/lons.

In [ ]:
if box_stream.data:
    element = gv.operation.project(box_stream.element, projection=ccrs.PlateCarree())
    xs, ys = element.array().T
    bbox = (xs[0], ys[0], xs[2], ys[1])
    print(bbox)

## Polygon Editing

The ``PolyEdit`` stream adds a bokeh tool to the source plot, which allows drawing, dragging and deleting vertices on polygons and making the drawn data available to Python. The tool supports the following actions:

**Show vertices**

    Double tap an existing patch or multi-line

**Add vertex**

    Double tap an existing vertex to select it, the tool will draw the next point, to add it tap in a new location.
    To finish editing and add a point double tap otherwise press the ESC key to cancel.

**Move vertex**

    Drag an existing vertex and let go of the mouse button to release it.

**Delete vertex**

    After selecting one or more vertices press BACKSPACE while the mouse cursor is within the plot area.

In [ ]:
%%opts Shape [width=900 height=500 tools=['box_select']] (alpha=0.5)

shapefile = './vicksburg_watershed/watershed_boundary.shp'
mask_shape = gv.Shape.from_shapefile(shapefile)[0]
vertex_stream = PolyEdit(source=mask_shape)
tiles * mask_shape

#### Saving out the edited shape

In [ ]:
%%opts Shape [width=600 height=400] (alpha=0.5)
from earthsim.tools import save_shapefile
if vertex_stream.data:
    edited_shape_fname = './vicksburg_watershed_edited/watershed_boundary.shp'
    save_shapefile(vertex_stream.data, edited_shape_fname, shapefile)
    mask_shape = gv.Shape.from_shapefile(edited_shape_fname)
mask_shape = mask_shape.opts() # Clear options to avoid adding edit tool
mask_shape.last.clone(crs=ccrs.GOOGLE_MERCATOR)

## Drawing Polygons

The ``PolyDraw`` tool allows drawing new polygons on a plot.

**Add patch/multi-line**

    Double tap to add the first vertex, then use tap to add each subsequent vertex, to finalize the draw action double tap to insert the final vertex or press the ESC key to stop drawing.

**Move patch/multi-line**

    Tap and drag an existing patch/multi-line, the point will be dropped once you let go of the mouse button.

**Delete patch/multi-line**

    Tap a patch/multi-line to select it then press BACKSPACE key while the mouse is within the plot area.

In [ ]:
%%opts Polygons [width=900 height=500] (fill_alpha=0 line_color='black') Path (line_width=5 color='black')
new_polys = gv.Polygons([], crs=ccrs.GOOGLE_MERCATOR)
new_paths = gv.Path([], crs=ccrs.GOOGLE_MERCATOR)
poly_stream = PolyDraw(source=new_polys)
path_stream = PolyDraw(source=new_paths)
tiles * new_polys * new_paths

In [ ]:
poly_stream.element.geom()

## Drawing and editing a polygon

By combining the polygon drawing and vertex editing tool we can both draw and edit polygons:

In [ ]:
%%opts Polygons [width=900 height=500] (fill_alpha=0.2 line_color='black')
new_polys = gv.Polygons([], crs=ccrs.GOOGLE_MERCATOR)
poly_stream = PolyDraw(source=new_polys)
vertex_stream = PolyEdit(source=new_polys)
tiles * new_polys

In [ ]:
poly_stream.element